# 动态网页的数据爬取（使用api）

## 本节课学习目的:

学会找api接口

学会解析json数据

复习函数知识，使用函数编写代码

## 什么是动态网页（动态网页的判断标准）

https://movie.douban.com/typerank?type_name=%E8%BF%90%E5%8A%A8&type=18&interval_id=100:90&action=

许多现代网站使用JavaScript来动态加载和渲染内容，这意味着当页面首次加载时，一些数据可能并未直接嵌入HTML中，而是通过后续的Ajax请求获取并添加到页面上。对于这种情况，传统的爬虫方法可能无法直接获取这些数据。

简单来说，就是网页新的内容出现，但是网址却没变。

Ajax（Asynchronous JavaScript and XML）是一种创建交互式网页应用的网页开发技术。它的主要特点是异步交互和浏览器页面局部刷新。在Ajax中，当客户端（通常是浏览器）需要与服务器进行交互时，可以发送一个请求到服务器，而无需等待服务器的响应就可以继续执行其他操作。一旦服务器响应完成，Ajax可以只更新页面的某一部分，而不是整个页面，这为用户提供了更好的体验。

处理Ajax动态加载数据的方法：
分析Ajax请求：使用浏览器的开发者工具（如Chrome的Network标签页）来观察页面加载过程中发出的Ajax请求，并找到包含所需数据的请求api。

模拟Ajax请求：在爬虫中直接使用requests库来模拟这些Ajax请求，获取数据。

使用Selenium或Puppeteer：这些工具可以模拟一个真实的浏览器环境，执行JavaScript代码，从而获取动态加载的内容。

## 寻找api接口的方法

打开”检查“，切换到”网络“，一般接口藏在XHR或者js里面，大多在XHR里面。刷新页面，下拉一两次页面。查看数据变化情况。使用预览，查看新增的数据在不在预览中出现。

## 以豆瓣电影排行榜为例，编写代码

### 总体步骤：找到接口，构造urls，解析json数据。写入文件。

In [ ]:
# 找到接口，测试是否需要登录才能拿到数据
# https://movie.douban.com/j/chart/top_list?type=18&interval_id=100%3A90&action=&start=20&limit=20
#上述网址直接粘贴到浏览器，直接能看到数据，则说明不需要登录。因此，不需要先获取cookie。

#然后分析接口规律，发现只有在start的值那里有变化，每翻一页数值+20.因此可以构造接口网址库，然后再做个循环即可。
# https://movie.douban.com/j/chart/top_list?type=18&interval_id=100%3A90&action=&start=0&limit=20
# https://movie.douban.com/j/chart/top_list?type=18&interval_id=100%3A90&action=&start=20&limit=20


In [2]:
# 生成豆瓣电影网址列表。

def generate_douban_urls(start, end, step):
    """
    :param start: 起始值
    :param end: 终止值
    :param step: 每次递增的步长
    :return: 生成的网址列表
    """
    base_url = "https://movie.douban.com/j/chart/top_list?type=18&interval_id=100%3A90&action=&start={}&limit=20"
    urls = [base_url.format(i) for i in range(start, end, step)]
    return urls



In [5]:
# 测试一下生成的网址列表
urls = generate_douban_urls(0, 60, 20)

# 打印前几个网址查看是否正确
for url in urls[:5]:
    print(url)

https://movie.douban.com/j/chart/top_list?type=18&interval_id=100%3A90&action=&start=0&limit=20
https://movie.douban.com/j/chart/top_list?type=18&interval_id=100%3A90&action=&start=20&limit=20
https://movie.douban.com/j/chart/top_list?type=18&interval_id=100%3A90&action=&start=40&limit=20


In [6]:
#从给定的URL列表中爬取数据。
import requests
import json

def fetch_data_from_urls(urls):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    
    for url in urls:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            movies_data = response.json()
            print("示例电影数据:", movies_data[0])
        else:
            print(f"请求失败，状态码：{response.status_code}")


In [7]:
#测试示例数据

fetch_data_from_urls(urls)

示例电影数据: {'rating': ['9.0', '45'], 'rank': 1, 'cover_url': 'https://img1.doubanio.com/view/photo/s_ratio_poster/public/p2401676338.jpg', 'is_playable': True, 'id': '26387939', 'types': ['剧情', '传记', '运动', '家庭'], 'regions': ['印度'], 'title': '摔跤吧！爸爸', 'url': 'https://movie.douban.com/subject/26387939/', 'release_date': '2017-05-05', 'actor_count': 11, 'vote_count': 1616259, 'score': '9.0', 'actors': ['阿米尔·汗', '法缇玛·萨那·纱卡', '桑亚·玛荷塔', '阿帕尔夏克提·库拉那', '沙克希·坦沃', '塞伊拉·沃西', '苏哈妮·巴特纳格尔', '里特维克·萨霍里', '吉里什·库卡尼', '什沙尔·夏尔马', '维万·巴特那'], 'is_watched': False}
示例电影数据: {'rating': ['8.3', '45'], 'rank': 21, 'cover_url': 'https://img2.doubanio.com/view/photo/s_ratio_poster/public/p1393286611.jpg', 'is_playable': True, 'id': '3023164', 'types': ['剧情', '传记', '运动'], 'regions': ['美国'], 'title': '点球成金', 'url': 'https://movie.douban.com/subject/3023164/', 'release_date': '2011-09-09', 'actor_count': 26, 'vote_count': 107668, 'score': '8.3', 'actors': ['布拉德·皮特', '乔纳·希尔', '菲利普·塞默·霍夫曼', '罗宾·怀特', '克里斯·帕拉特', '鲍勃·科斯塔斯', '

# json格式介绍
JSON（JavaScript Object Notation）是一种轻量级的数据交换格式，易于人阅读和编写，同时也易于机器解析和生成。JSON的设计意图是作为一种简单的数据格式来交换数据。其格式是完全独立于语言的文本格式，但是它使用了类似于C语言家族（包括C, C++, C#, Java, JavaScript, Perl, Python等）的习惯，这些特性使JSON成为理想的数据交换语言。

JSON结构由以下两种结构类型组成：

对象（Object）：对象在JSON中是由键值对（key/value pairs）构成的无序集合。它被包裹在花括号{}中，如{"name": "John", "age": 30}。键名（key）是字符串，每个键值对之间用逗号,分隔。
数组（Array）：数组是值（value）的有序集合。数组在JSON中被包裹在方括号[]中，如["Apple", "Banana", "Cherry"]。数组中的值可以是数字、字符串、布尔值、对象或其他数组等。

JSON的基本规则包括：

数据在名称/值对中：字段名称（键）被引号（通常是双引号）包围，后面跟着一个冒号，然后是值。
数据由逗号分隔：对象中的多个名称/值对由逗号分隔，数组中的多个值也由逗号分隔。
大括号{}保存对象，方括号[]保存数组。

JSON支持的数据类型包括：数值（整数或浮点数）、字符串（在双引号中）、布尔值（true或false）、数组（在方括号中）、对象（在大括号中）和null。

JSON的使用场景

JSON广泛用于Web应用程序之间的数据交换。由于它的轻量级和易于解析的特性，JSON非常适合从服务器发送数据到客户端，客户端通过JavaScript（或其他语言）进行解析并显示数据。此外，许多编程语言都提供了将本地数据结构转换为JSON格式字符串的方法，以及将JSON格式字符串解析为本地数据结构的方法，使得JSON成为跨平台和跨语言交换数据的理想格式。

在Python中，json模块提供了一套简单的方法来编码和解码JSON数据。它允许将Python对象转换为JSON格式的字符串，以及将JSON格式的字符串解析回Python对象。这个过程通常被称为序列化（将对象转换为字符串）和反序列化（将字符串转换为对象）。

### 导入JSON模块

In [ ]:
import json

### JSON编码（序列化）
将Python对象转换成JSON字符串的过程称为编码（序列化）。主要使用json.dumps()和json.dump()方法。

In [ ]:
#json.dumps(obj): 将Python对象obj转换为JSON格式的字符串。

import json

data = {
    "name": "John Doe",
    "age": 30,
    "is_employee": True
}

json_string = json.dumps(data)
print(json_string)


In [ ]:
# json.dump(obj, file): 将Python对象obj转换为JSON格式并写入到file对象中。

import json

data = {
    "name": "John Doe",
    "age": 30,
    "is_employee": True
}

with open('data.json', 'w') as json_file:
    json.dump(data, json_file)


### JSON解码（反序列化）
将JSON格式的字符串解析回Python对象的过程称为解码（反序列化）。主要使用json.loads()和json.load()方法。

In [ ]:
# json.loads(s): 将JSON格式的字符串s解析为Python对象。

import json

json_string = '{"name": "John Doe", "age": 30, "is_employee": true}'

data = json.loads(json_string)
print(data)


In [ ]:
# json.load(file): 从file对象中读取JSON格式的字符串并解析为Python对象。

import json

with open('data.json', 'r') as json_file:
    data = json.load(json_file)
    print(data)


### 注意事项
JSON中的true, false, 和 null在Python中分别对应为True, False, 和 None。

JSON编码支持的Python数据类型包括dict, list, tuple, str, int, float, True, False, 和 None。尝试将其他类型的对象（如自定义类）序列化为JSON时，需要提供转换方法或使用自定义编码器。

对于非标准的数据类型，你可能需要自定义序列化和反序列化的方法，以确保正确地处理这些数据。

In [8]:
## 解析获取到的json数据
def parse_movie_data(movie_data):
    """
    从电影数据中解析title、url和score。
    
    :param movie_data: 单部电影的数据
    :return: 解析后的数据
    """
    title = movie_data.get('title')
    #title = movie_data['title']  # 如果键'title'不存在，这里会引发KeyError。如果是嵌套字典，那么可以使用这个方法，嵌套查询到深层的值。
    url = movie_data.get('url')
    score = movie_data.get('score')
    return title, url, score

In [11]:
#更新代码，将parse_movie_data函数并入到fetch_data_from_urls函数中
import requests
import json

def fetch_data_from_urls(urls):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    
    for url in urls:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            movies_data = response.json()
            for movie in movies_data:
                # 使用parse_movie_data函数解析数据
                title, url, score = parse_movie_data(movie)
                print(f"{title}, {score}, {url}")
        else:
            print(f"请求失败，状态码：{response.status_code}")

In [12]:
#验证上述代码
fetch_data_from_urls(urls)

摔跤吧！爸爸, 9.0, https://movie.douban.com/subject/26387939/
灌篮高手, 8.9, https://movie.douban.com/subject/35315950/
勇士, 8.9, https://movie.douban.com/subject/3217169/
百万美元宝贝, 8.8, https://movie.douban.com/subject/1309016/
弱点, 8.7, https://movie.douban.com/subject/3552028/
洛奇, 8.8, https://movie.douban.com/subject/1295742/
追梦赤子心, 8.8, https://movie.douban.com/subject/1294753/
极速风流, 8.7, https://movie.douban.com/subject/6803494/
铁拳男人, 8.8, https://movie.douban.com/subject/1418519/
坏孩子的天空, 8.6, https://movie.douban.com/subject/1299062/
极速车王, 8.5, https://movie.douban.com/subject/6538866/
卡特教练, 8.6, https://movie.douban.com/subject/1309017/
光荣之路, 8.6, https://movie.douban.com/subject/1457219/
世上最快的印第安摩托, 8.9, https://movie.douban.com/subject/1449711/
光辉岁月, 8.6, https://movie.douban.com/subject/1292943/
我在雨中等你, 8.4, https://movie.douban.com/subject/3882427/
铁甲钢拳, 8.3, https://movie.douban.com/subject/1972729/
百元之恋, 8.3, https://movie.douban.com/subject/25761178/
愤怒的公牛, 8.4, https://movie.douban.c

In [ ]:
#将收集到的电影信息保存到csv文件中
import requests
import json
import csv

#写入csv函数
def save_data_to_csv(data, filename='movies_data.csv'):
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Title', 'Score', 'URL'])
        for movie in data:
            writer.writerow([movie['title'], movie['score'], movie['url']])
    print(f"数据已保存到{filename}")

# 生成豆瓣电影网址列表。
def generate_douban_urls(start, end, step):
    """
    :param start: 起始值
    :param end: 终止值
    :param step: 每次递增的步长
    :return: 生成的网址列表
    """
    base_url = "https://movie.douban.com/j/chart/top_list?type=18&interval_id=100%3A90&action=&start={}&limit=20"
    urls = [base_url.format(i) for i in range(start, end, step)]
    return urls

## 解析获取到的json数据
def parse_movie_data(movie_data):
    """
    从电影数据中解析title、url和score。
    
    :param movie_data: 单部电影的数据
    :return: 解析后的数据
    """
    title = movie_data.get('title')
    url = movie_data.get('url')
    score = movie_data.get('score')
    return title, url, score

#从给定的URL列表中爬取数据。
def fetch_data_from_urls(urls):
    all_movies_data = []  # 初始化列表以收集所有电影数据
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    
    for url in urls:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            movies_data = response.json()
            for movie in movies_data:
                title, url, score = parse_movie_data(movie)
                all_movies_data.append({'title': title, 'url': url, 'score': score})
        else:
            print(f"请求失败，状态码：{response.status_code}")
    
    # 在完成所有请求后，将数据保存到CSV文件
    save_data_to_csv(all_movies_data)

#运行程序
urls = generate_douban_urls(0, 60, 20)  
fetch_data_from_urls(urls)

In [ ]:
#将收集到的电影信息保存到Excel文件中
import pandas as pd
import requests
import json

#将爬取的数据保存到Excel文件中
def save_data_to_excel(data, filename='movies_data.xlsx'):
    """
    :param data: 要保存的数据列表，每个元素是一个包含电影信息的字典。
    :param filename: Excel文件的名称。
    """
    # 将数据转换为DataFrame
    df = pd.DataFrame(data)
    # 保存DataFrame到Excel，确保安装了openpyxl
    df.to_excel(filename, index=False, engine='openpyxl')
    print(f"数据已保存到{filename}")

# 生成豆瓣电影网址列表。
def generate_douban_urls(start, end, step):
    """
    :param start: 起始值
    :param end: 终止值
    :param step: 每次递增的步长
    :return: 生成的网址列表
    """
    base_url = "https://movie.douban.com/j/chart/top_list?type=18&interval_id=100%3A90&action=&start={}&limit=20"
    urls = [base_url.format(i) for i in range(start, end, step)]
    return urls

## 解析获取到的json数据
def parse_movie_data(movie_data):
    """
    从电影数据中解析title、url和score。
    
    :param movie_data: 单部电影的数据
    :return: 解析后的数据
    """
    title = movie_data.get('title')
    url = movie_data.get('url')
    score = movie_data.get('score')
    return title, url, score

#从给定的URL列表中爬取数据。
def fetch_data_from_urls(urls):
    movies_data = []  # 初始化空列表以收集电影数据
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    
    for url in urls:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            movies_json = response.json()
            for movie in movies_json:
                # 使用parse_movie_data函数解析数据
                title, url, score = parse_movie_data(movie)
                movies_data.append({'title': title, 'url': url, 'score': score})
        else:
            print(f"请求失败，状态码：{response.status_code}")
            
    # 在完成所有请求后保存数据到Excel
    save_data_to_excel(movies_data)

#运行程序
urls = generate_douban_urls(0, 60, 20)  
fetch_data_from_urls(urls)

# 作业
获取该链接下：https://movie.douban.com/chart，右侧分类排行榜某一分类电影的所有结果,包含电影名称、网址、评分、国别、发布时间等内容。

每人一个分类：剧情 喜剧 动作 爱情 科幻 动画 悬疑 惊悚 恐怖 纪录片 短片 情色 音乐 歌舞 家庭 儿童 传记 历史 战争 犯罪 西部 奇幻 冒险 灾难 武侠 古装 运动 黑色电影

班长进行统计如何分。
将代码和获取到的数据提交到超星学习通。